In [7]:
import openml
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [8]:
datasets = [1485, 1590, 1515, 1457, 1475, 1468, 1486, 1489, 23512, 23517, 4541, 4534, 4538, 4134, 4135, 40978, 40996, 41027, 40981, 40982, 40983, 40984, 40701, 40670, 40685, 40900,  1111, 42732, 42733, 42734, 40498, 41161, 41162, 41163, 41164, 41165, 41166, 41167, 41168, 41169, 41142, 41143, 41144, 41145, 41146, 41147, 41150, 41156, 41157, 41158,  41159, 41138, 54, 181, 188, 1461, 1494, 1464, 12, 23, 3, 1487, 40668, 1067, 1049, 40975, 31]
rows = []
for openmlid in tqdm(datasets):
    
    # load dataset object
    dataset = openml.datasets.get_dataset(openmlid)
    
    # analyze columns of data
    dfDataset = dataset.get_data()[0]
    types = dfDataset.dtypes
    cnt_normalized = 0
    cnt_zero_mean = 0
    cnt_one_std = 0
    cnt_numerical = 0
    for i, col in enumerate(dfDataset.columns):
        if "float" in str(types[i]):
            cnt_numerical += 1
            vals = dfDataset[col].values
            is_normalized = np.round(min(vals), 3) == 0 and np.round(max(vals), 3) == 1
            is_zero_mean = np.round(np.mean(vals), 3) == 0
            is_one_std = np.round(np.std(vals), 3) == 1
            if is_normalized:
                cnt_normalized += 1
            if is_zero_mean:
                cnt_zero_mean += 1
            if is_one_std:
                cnt_one_std += 1
    if cnt_numerical > 0:
        feature_stats_entries = [str(v) + "\%" for v in np.round(100 * np.array([cnt_normalized / cnt_numerical, cnt_zero_mean / cnt_numerical, cnt_one_std / cnt_numerical]), 0).astype(int)]
    else:
        feature_stats_entries = 3 * ["n/a"]
        
    row = [openmlid, dataset.name[:20].replace("_", "\\_"), int(dataset.qualities["NumberOfInstances"]), len(dataset.features) - 1, int(dataset.qualities["NumberOfNumericFeatures"]), int(dataset.qualities["NumberOfClasses"]), str(int(dataset.qualities["MinorityClassPercentage"])) + "\%", str(int(dataset.qualities["MajorityClassPercentage"])) + "\%", str(int(dataset.qualities["PercentageOfMissingValues"])) + "\%"] + feature_stats_entries
    rows.append(row)
dfDatasets = pd.DataFrame(rows, columns=["openmlid", "name", "instances", "features", "numeric features", "classes", "min \%", "maj \%", "\% missing", "\% [0,1]", "\% $\mu = 0$", "\% $\sigma = 1$"]).sort_values("openmlid")

In [15]:
dfDatasets[dfDatasets["\% missing"] != "0\%"]["openmlid"].values

array([  188,  1111, 41138, 41147, 41162, 42734])

In [14]:
dfDatasets[dfDatasets["openmlid"] == 41157]

,openmlid,name,instances,features,numeric features,classes,min \%,maj \%,\% missing,"\% [0,1]",\% $\mu = 0$,\% $\sigma = 1$
48,41157,arcene,100,10000,10000,2,44\%,56\%,0\%,0\%,1\%,0\%
